# 3/ Deploying our frontend App with Lakehouse Applications


Mosaic AI Agent Evaluation review app is used for collecting stakeholder feedback during your development process.

You still need to deploy your own front end application!

Let's leverage Databricks Lakehouse Applications to build and deploy our first, simple chatbot frontend app. 

<img src="https://github.com/databricks-demos/dbdemos-resources/blob/main/images/product/chatbot-rag/rag-frontend-app.png?raw=true" width="1200px">


<div style="background-color: #d4e7ff; padding: 10px; border-radius: 15px;">
<strong>Note:</strong> Lakehouse apps are in preview, reach-out your Databricks Account team for more details and to enable it.
</div>

<!-- Collect usage data (view). Remove it to disable collection or disable tracker during installation. View README for more details.  -->
<img width="1px" src="https://ppxrzfxige.execute-api.us-west-2.amazonaws.com/v1/analytics?category=data-science&org_id=341332174749405&notebook=%2F02-simple-app%2F03-Deploy-Frontend-Lakehouse-App&demo_name=llm-rag-chatbot&event=VIEW&path=%2F_dbdemos%2Fdata-science%2Fllm-rag-chatbot%2F02-simple-app%2F03-Deploy-Frontend-Lakehouse-App&version=1">

In [0]:
%pip install --quiet -U mlflow[databricks] databricks-sdk==0.59.0
dbutils.library.restartPython()

In [0]:
%run ../_resources/00-init $reset_all_data=false

## Add your application configuration

Lakehouse apps let you work with any python framework. For our small demo, we will create a small configuration file containing the model serving endpoint name used for our demo and save it in the `chatbot_app/app.yaml` file.

In [0]:
MODEL_NAME = "dbdemos_rag_demo"
endpoint_name = f'agents_{catalog}-{db}-{MODEL_NAME}'[:60]

# Our frontend application will hit the model endpoint we deployed.
# Because dbdemos let you change your catalog and database, let's make sure we deploy the app with the proper endpoint name
yaml_app_config = {"command": ["uvicorn", "main:app", "--workers", "1"],
                    "env": [{"name": "MODEL_SERVING_ENDPOINT", "value": endpoint_name}]
                  }
try:
    with open('chatbot_app/app.yaml', 'w') as f:
        yaml.dump(yaml_app_config, f)
except:
    print('pass to work on build job')

## Let's now create our chatbot application using Gradio

In [0]:
%%writefile chatbot_app/main.py
from fastapi import FastAPI
import gradio as gr
import os
from gradio.themes.utils import sizes
from databricks.sdk import WorkspaceClient
from databricks.sdk.service.serving import ChatMessage, ChatMessageRole

app = FastAPI()

# your endpoint will directly be setup with proper permissions when you deploy your app
w = WorkspaceClient()
available_endpoints = [x.name for x in w.serving_endpoints.list()]


def respond(message, history, dropdown):
    if len(message.strip()) == 0:
        return "ERROR the question should not be empty"
    try:
        messages = []
        if history:
            for human, assistant in history:
                messages.append(ChatMessage(content=human, role=ChatMessageRole.USER))
                messages.append(
                    ChatMessage(content=assistant, role=ChatMessageRole.ASSISTANT)
                )
        messages.append(ChatMessage(content=message, role=ChatMessageRole.USER))
        response = w.serving_endpoints.query(
            name=dropdown,
            messages=messages,
            temperature=1.0,
            stream=False,
        )
    except Exception as error:
        return f"ERROR requesting endpoint {dropdown}: {error}"
    return response.choices[0].message.content


theme = gr.themes.Soft(
    text_size=sizes.text_sm,
    radius_size=sizes.radius_sm,
    spacing_size=sizes.spacing_sm,
)

demo = gr.ChatInterface(
    respond,
    chatbot=gr.Chatbot(
        show_label=False, container=False, show_copy_button=True, bubble_full_width=True
    ),
    textbox=gr.Textbox(placeholder="What is RAG?", container=False, scale=7),
    title="Databricks App RAG demo - Chat with your Databricks assistant",
    description="This chatbot is a demo example for the dbdemos llm chatbot. <br>It answers with the help of Databricks Documentation saved in a Knowledge database.<br/>This content is provided as a LLM RAG educational example, without support. It is using DBRX, can hallucinate and should not be used as production content.<br>Please review our dbdemos license and terms for more details.",
    examples=[
        ["What is DBRX?"],
        ["How can I start a Databricks cluster?"],
        ["What is a Databricks Cluster Policy?"],
        ["How can I track billing usage on my workspaces?"],
    ],
    cache_examples=False,
    theme=theme,
    retry_btn=None,
    undo_btn=None,
    clear_btn="Clear",
    additional_inputs=gr.Dropdown(
        choices=available_endpoints,
        value=os.environ["MODEL_SERVING_ENDPOINT"],
        label="Serving Endpoint",
    ),
    additional_inputs_accordion="Settings",
)

demo.queue(default_concurrency_limit=100)
app = gr.mount_gradio_app(app, demo, path="/")

## Deploying our application

Our application is made of 2 files under the `chatbot_app` folder:
- `main.py` containing our python code
- `app.yaml` containing our configuration

All we now have to do is call the API to create a new app and deploy using the `chatbot_app` path:

In [0]:
from databricks.sdk import WorkspaceClient
from databricks.sdk.service.apps import App, AppResource, AppResourceServingEndpoint, AppResourceServingEndpointServingEndpointPermission, AppDeployment

w = WorkspaceClient()

app_name = "dbdemos-rag-chatbot-app"

Lakehouse apps come with an auto-provisioned Service Principal. Let's grant this Service Principal access to our model endpoint before deploying...

In [0]:
serving_endpoint = AppResourceServingEndpoint(name=endpoint_name,
                                              permission=AppResourceServingEndpointServingEndpointPermission.CAN_QUERY
                                              )

rag_endpoint = AppResource(name="rag-endpoint", serving_endpoint=serving_endpoint) 

rag_app = App(name=app_name, 
              description="Your Databricks assistant", 
              default_source_code_path=os.path.join(os.getcwd(), 'chatbot_app'),
              resources=[rag_endpoint])
try:
  app_details = w.apps.create_and_wait(app=rag_app)
  print(app_details)
except Exception as e:
  if "already exists" in str(e):
    print("App already exists, you can deploy it")
  else:
    raise e

Once the app is created, we can (re)deploy the code as following:

In [0]:
deployment = AppDeployment(
  source_code_path=os.path.join(os.getcwd(), 'chatbot_app')
)

app_details = w.apps.deploy_and_wait(app_name=app_name, app_deployment=deployment)

In [0]:
#Let's access the application
w.apps.get(name=app_name).url

## Your Lakehouse app is ready and deployed!

<img src="https://github.com/databricks-demos/dbdemos-resources/blob/main/images/product/chatbot-rag/rag-gradio-app.png?raw=true" width="750px" style="float: right; margin-left:10px">

Open the UI to start requesting your chatbot.

As improvement, we could improve our chatbot UI to provide feedback and send it to Mosaic AI Quality Labs, so that bad answers can be reviewed and improved.

## Conclusion

We saw how Databricks provides an end to end platform: 
- Building and deploying an endpoint
- Buit-in solution to review, analyze and improve our chatbot
- Deploy front-end genAI application with lakehouse apps!

## Next: ready to take it to a next level?

Open the [03-advanced-app/01-PDF-Advanced-Data-Preparation]($../03-advanced-app/01-PDF-Advanced-Data-Preparation) notebook series to learn more about unstructured data, advanced chain, model evaluation and monitoring.